# Selective flow constraints with [PowerSimulations.jl](https://github.com/NREL/PowerSimulations.jl)

**Originally Contributed by**: Clayton Barrows and Sourabh Dalvi

## Introduction

The [Operations Problems example]](../../notebook/PowerSimulations_examples/sequential_simulations.ipynb)
shows the basic building blocks of building optimization problems with PowerSimulations.jl.
This example shows how to customize the enforcement of branch flow constraints as is common
when trying to build large scale simulations.

## Dependencies

In [1]:
using SIIPExamples

### Modeling Packages

In [2]:
using PowerSystems
using PowerSimulations

### Optimization packages
For this simple example, we can use the Cbc solver with a relatively relaxed tolerance.

In [3]:
using Cbc #solver
solver = optimizer_with_attributes(Cbc.Optimizer, "logLevel" => 1, "ratioGap" => 0.5)

MathOptInterface.OptimizerWithAttributes(Cbc.Optimizer, Pair{MathOptInterface.AbstractOptimizerAttribute,Any}[MathOptInterface.RawParameter("logLevel") => 1, MathOptInterface.RawParameter("ratioGap") => 0.5])

### Create a `System` from RTS-GMLC data
We can just use the
[parsing tabular data example.](../../notebook/PowerSystems_examples/parse_tabulardata.jl)

In [4]:
pkgpath = dirname(dirname(pathof(SIIPExamples)))
include(joinpath(pkgpath, "test", "PowerSystems_examples", "parse_tabulardata.jl"))

┌ Info: Parsing csv files in Reserves ...
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/m5IPy/src/parsers/power_system_table_data.jl:130
┌ Info: Parsing csv data in DAY_AHEAD_regional_Flex_Down.csv ...
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/m5IPy/src/parsers/power_system_table_data.jl:134
┌ Info: Parsing csv data in DAY_AHEAD_regional_Flex_Up.csv ...
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/m5IPy/src/parsers/power_system_table_data.jl:134
┌ Info: Parsing csv data in DAY_AHEAD_regional_Reg_Down.csv ...
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/m5IPy/src/parsers/power_system_table_data.jl:134
┌ Info: Parsing csv data in DAY_AHEAD_regional_Reg_Up.csv ...
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/m5IPy/src/parsers/power_system_table_data.jl:134
┌ Info: Parsing csv data in DAY_AHEAD_regional_Spin_Up_R1.csv ...
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/m5IPy/src/parsers/pow

,ConcreteType,SuperTypes,Count
,String,String,Int64
1,Area,AggregationTopology <: Topology <: Component <: PowerSystemType <: InfrastructureSystemsType <: Any,3
2,Bus,Topology <: Component <: PowerSystemType <: InfrastructureSystemsType <: Any,73
3,GenericBattery,Storage <: StaticInjection <: Device <: Component <: PowerSystemType <: InfrastructureSystemsType <: Any,1
4,HVDCLine,DCBranch <: Branch <: Device <: Component <: PowerSystemType <: InfrastructureSystemsType <: Any,1
5,HydroDispatch,HydroGen <: Generator <: StaticInjection <: Device <: Component <: PowerSystemType <: InfrastructureSystemsType <: Any,20
6,Line,ACBranch <: Branch <: Device <: Component <: PowerSystemType <: InfrastructureSystemsType <: Any,104
7,LoadZone,AggregationTopology <: Topology <: Component <: PowerSystemType <: InfrastructureSystemsType <: Any,21
8,PowerLoad,StaticLoad <: ElectricLoad <: StaticInjection <: Device <: Component <: PowerSystemType <: InfrastructureSystemsType <: Any,51
9,RenewableDispatch,RenewableGen <: Generator <: StaticInjection <: Device <: Component <: PowerSystemType <: InfrastructureSystemsType <: Any,30


### Selecting flow limited lines
Since PowerSimulations will apply constraints by component type (e.g. Line), we need to
change the component type of the lines on which we want to enforce flow limits. So, let's
change the device type of certain branches from Line to MonitoredLine differentiate
treatment when we build the model. Here, we can select inter-regional lines, or lines
above a voltage threshold.

In [5]:
for line in get_components(Line, sys)
    if (get_basevoltage(get_from(get_arc(line))) >= 230.0) &&
       (get_basevoltage(get_to(get_arc(line))) >= 230.0)
        #if get_area(get_from(get_arc(line))) != get_area(get_to(get_arc(line)))
        @info "Changing $(get_name(line)) to MonitoredLine"
        convert_component!(MonitoredLine, line, sys)
    end
end

┌ Info: Changing C31-2 to MonitoredLine
└ @ Main.##426 string:5
┌ Info: Changing B26 to MonitoredLine
└ @ Main.##426 string:5
┌ Info: Changing A32-2 to MonitoredLine
└ @ Main.##426 string:5
┌ Info: Changing CA-1 to MonitoredLine
└ @ Main.##426 string:5
┌ Info: Changing B34 to MonitoredLine
└ @ Main.##426 string:5
┌ Info: Changing B19 to MonitoredLine
└ @ Main.##426 string:5
┌ Info: Changing B31-2 to MonitoredLine
└ @ Main.##426 string:5
┌ Info: Changing C30 to MonitoredLine
└ @ Main.##426 string:5
┌ Info: Changing B27 to MonitoredLine
└ @ Main.##426 string:5
┌ Info: Changing A21 to MonitoredLine
└ @ Main.##426 string:5
┌ Info: Changing A32-1 to MonitoredLine
└ @ Main.##426 string:5
┌ Info: Changing B29 to MonitoredLine
└ @ Main.##426 string:5
┌ Info: Changing A18 to MonitoredLine
└ @ Main.##426 string:5
┌ Info: Changing C24 to MonitoredLine
└ @ Main.##426 string:5
┌ Info: Changing C28 to MonitoredLine
└ @ Main.##426 string:5
┌ Info: Changing A29 to MonitoredLine
└ @ Main.##426 string:5

## Build an `OperationsProblem`

In [6]:
uc_prob =
    UnitCommitmentProblem(sys, optimizer = solver, horizon = 24, slack_variables = true)

┌ Warning: The data doesn't include devices of type PowerSystems.InterruptibleLoad, consider changing the device models
└ @ PowerSimulations /Users/cbarrows/Documents/repos/PowerSimulations.jl/src/devices_models/device_constructors/common/constructor_validations.jl:4
┌ Warning: The data doesn't include devices of type PowerSystems.HydroEnergyReservoir, consider changing the device models
└ @ PowerSimulations /Users/cbarrows/Documents/repos/PowerSimulations.jl/src/devices_models/device_constructors/common/constructor_validations.jl:4



Operations Problem Specification

  transmission:  PowerSimulations.CopperPlatePowerModel
  devices: 
      ILoads:
        device_type = PowerSystems.InterruptibleLoad
        formulation = PowerSimulations.InterruptiblePowerLoad
      HydroROR:
        device_type = PowerSystems.HydroDispatch
        formulation = PowerSimulations.HydroFixed
      Generators:
        device_type = PowerSystems.ThermalStandard
        formulation = PowerSimulations.ThermalBasicUnitCommitment
      DistRE:
        device_type = PowerSystems.RenewableFix
        formulation = PowerSimulations.RenewableFixed
      Hydro:
        device_type = PowerSystems.HydroEnergyReservoir
        formulation = PowerSimulations.HydroDispatchReservoirFlow
      Loads:
        device_type = PowerSystems.PowerLoad
        formulation = PowerSimulations.StaticPowerLoad
      RE:
        device_type = PowerSystems.RenewableDispatch
        formulation = PowerSimulations.RenewableFullDispatch
  branches: 
      T:
        

The above function defaults to a basic `CopperPlatePowerModel`, ror now, let's just
choose a standard DCOPF (B-theta) formulation.

In [7]:
set_transmission_model!(uc_prob, DCPPowerModel) #TODO: rm this and add network = DCPPowerModel to above when PSIMA-138 is tagged

┌ Warning: The data doesn't include devices of type PowerSystems.InterruptibleLoad, consider changing the device models
└ @ PowerSimulations /Users/cbarrows/Documents/repos/PowerSimulations.jl/src/devices_models/device_constructors/common/constructor_validations.jl:4
┌ Warning: The data doesn't include devices of type PowerSystems.HydroEnergyReservoir, consider changing the device models
└ @ PowerSimulations /Users/cbarrows/Documents/repos/PowerSimulations.jl/src/devices_models/device_constructors/common/constructor_validations.jl:4


Let's change the formulation of the `Line` components to an unbounded flow formulation.
This formulation still enforces Kirchoff's laws, but does not apply flow constraints.

In [8]:
set_branch_model!(uc_prob, :L, DeviceModel(Line, StaticLineUnbounded))

┌ Warning: The data doesn't include devices of type PowerSystems.InterruptibleLoad, consider changing the device models
└ @ PowerSimulations /Users/cbarrows/Documents/repos/PowerSimulations.jl/src/devices_models/device_constructors/common/constructor_validations.jl:4
┌ Warning: The data doesn't include devices of type PowerSystems.HydroEnergyReservoir, consider changing the device models
└ @ PowerSimulations /Users/cbarrows/Documents/repos/PowerSimulations.jl/src/devices_models/device_constructors/common/constructor_validations.jl:4


Notice that there is no entry for `MonitoredLine` branches. So, let's add one.

In [9]:
construct_device!(uc_prob, :ML, DeviceModel(MonitoredLine, StaticLine))

Solve the relaxed problem

In [10]:
solve!(uc_prob)

┌ Warning: SolveTime() property not supported by COIN Branch-and-Cut (Cbc)
└ @ PowerSimulations /Users/cbarrows/Documents/repos/PowerSimulations.jl/src/core/operations_problem.jl:523


,Time,A9,A12-1,C9,C4,B11,A11
,Dates…,Float64,Float64,Float64,Float64,Float64,Float64
1,2020-01-01T00:00:00,0.0862704,-0.686343,0.0145026,-0.177702,-0.48363,-0.649821
2,2020-01-01T01:00:00,0.101633,-0.73454,-0.245661,-0.222512,-0.474972,-0.737234
3,2020-01-01T02:00:00,0.0727287,-0.663596,0.0381404,-0.139876,-0.473595,-0.596269
4,2020-01-01T03:00:00,0.0711287,-0.702078,-0.0345638,-0.117662,-0.473005,-0.641993
5,2020-01-01T04:00:00,0.0458819,-0.753463,-0.0264039,-0.181012,-0.482056,-0.662129
6,2020-01-01T05:00:00,0.34243,-0.782709,-0.0285196,-0.154291,-0.507044,-0.674597
7,2020-01-01T06:00:00,0.383608,-0.790939,-0.138321,-0.166168,-0.540886,-0.670481
8,2020-01-01T07:00:00,0.611543,-0.829465,-0.393764,-0.206684,-0.535377,-0.790478
9,2020-01-01T08:00:00,0.347939,-0.813237,-0.568783,-0.248586,-0.528097,-0.756276


---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*